<a href="https://colab.research.google.com/github/VIVEKPATIL12/PotholeDetectionHoldData/blob/main/potholesDetecrionHoldData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
# Define Friction Coefficients
friction_coefficients = {
    "Wet Asphalt" :"0.40-0.50",
    "Black Ice": "0.005-0.02",
    "Wet Cobblestone" : "0.30-0.40",
    "Wet Concrete" : "0.40-0.50",
    "Wet Gravel" : "0.30-0.40",
    "Heavy Snow": "0.15-0.30",
    "Wet Heavy" :  "0.30-0.50",
    "Light Snow": "0.05-0.20",
    "Muddy Wet" :   "0.20-0.30",
    "Packed Snow": "0.05-0.15",
    "Refrozen Ice": "0.30-0.40",
    "Sleet": "0.50",
    "Slush": "0.35-0.40",
    "Dry Asphalt" : "0.70-0.80",
    "Dry Cobblestone" : "0.60-0.70",
    "Dry Concrete" :  "0.70-0.80",
    "Dry Mud" :  "0.50-0.60",
    "Dry Gravel" : "0.60-0.70"
}

# Vehicle parameters
VEHICLE_MASS = 1500.0  # kg
GRAVITY = 9.81         # m/s^2
TIRE_RADIUS = 0.3      # meters (approximately)

# Kalman filter variables
Q = 0.0001  # Process noise covariance
R = 0.01    # Measurement noise covariance
A = 1.0     # State transition matrix (friction coefficient evolution)
H = 1.0     # Measurement matrix (direct observation)

# Kalman filter state and covariance
friction_estimate = 0.8  # Initial friction estimate (initial guess)
P = 1.0                  # Initial covariance (uncertainty)


vehicle_speed = 20.0  # m/s (example value)
tire_angular_velocity = 10.0  # rad/s (example value)


# Function to calculate slip ratio
def calculate_slip_ratio(vehicle_speed, tire_angular_velocity):
    tire_speed = tire_angular_velocity * TIRE_RADIUS  # Tire surface speed (m/s)
    if vehicle_speed == 0:
        return 0  # Avoid division by zero
    return (tire_speed - vehicle_speed) / vehicle_speed

# Function to update the Kalman filter state
def kalman_filter(measured_mu):
    global friction_estimate, P
    # Prediction step
    friction_estimate = A * friction_estimate
    P = A * P * A + Q

    # Update step
    K = P * H / (H * P * H + R)
    friction_estimate = friction_estimate + K * (measured_mu - H * friction_estimate)
    P = (1 - K * H) * P

    return friction_estimate

class_names = ['Dry Asphalt', 'Dry Asphalt', 'Dry Asphalt', 'Dry Asphalt', 'Dry Asphalt', 'Dry Asphalt',
           'Dry Asphalt', 'Dry Asphalt', 'Dry Asphalt', 'Dry Asphalt', 'Dry Asphalt', 'Dry Asphalt',
           'Dry Asphalt', 'Dry Asphalt', 'Dry Asphalt', 'Dry Asphalt', 'Dry Asphalt', 'Dry Asphalt']
import os
import cv2
import math
import time
import shutil
import geocoder
import numpy as np
import pandas as pd
import tensorflow as tf
# from google.colab.patches import cv2_imshow
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


# class_names = ['Dry Asphalt', 'Wet Gravel', 'Light Snow', 'Sleet']

class_names = ['Dry Asphalt', 'Dry Asphalt', 'Dry Asphalt', 'Dry Asphalt', 'Dry Asphalt', 'Dry Asphalt',
           'Dry Asphalt', 'Dry Asphalt', 'Dry Asphalt', 'Dry Asphalt', 'Dry Asphalt', 'Dry Asphalt',
           'Dry Asphalt', 'Dry Asphalt', 'Dry Asphalt', 'Dry Asphalt', 'Dry Asphalt', 'Dry Asphalt']

# class_names=['Muddy Wet', 'Muddy Wet', 'Muddy Wet', 'Muddy Wet', 'Muddy Wet', 'Muddy Wet',
#            'Muddy Wet', 'Muddy Wet', 'Muddy Wet', 'Muddy Wet', 'Muddy Wet', 'Muddy Wet',
#            'Muddy Wet', 'Muddy Wet', 'Muddy Wet', 'Muddy Wet', 'Muddy Wet', 'Muddy Wet']

# class_names= ['Black Ice','Dry Asphalt','Dry Cobblestone','Dry Concrete','Dry Gravel','Dry Mud',
#                'Heavy Snow','Light Snow','Muddy Wet','Packed Snow','Refrozen Ice','Sleet','Slush',
#                'Wet Asphalt','Wet Cobblestone','Wet Concrete','Wet Gravel','Wet Heavy']

In [ ]:



model=load_model("/home/vivekp22/Downloads/Road_Surface_Classification_Video_Based (1).keras")
# Define Friction Coefficients
friction_coefficients = {
    "Wet Asphalt" :"0.40-0.50",
    "Black Ice": "0.005-0.02",
    "Wet Cobblestone" : "0.30-0.40",
    "Wet Concrete" : "0.40-0.50",
    "Wet Gravel" : "0.30-0.40",
    "Heavy Snow": "0.15-0.30",
    "Wet Heavy" :  "0.30-0.50",
    "Light Snow": "0.05-0.20",
    "Muddy Wet" :   "0.20-0.30",
    "Packed Snow": "0.05-0.15",
    "Refrozen Ice": "0.30-0.40",
    "Sleet": "0.50",
    "Slush": "0.35-0.40",
    "Dry Asphalt" : "0.70-0.80",
    "Dry Cobblestone" : "0.60-0.70",
    "Dry Concrete" :  "0.70-0.80",
    "Dry Mud" :  "0.50-0.60",
    "Dry Gravel" : "0.60-0.70"
}


# Constants for Kalman Filter and Pacejka
NUM_WHEELS = 4
VEHICLE_MASS = 1500.0
GRAVITY = 9.81
Q = 0.0001  # Process noise
R = 0.01    # Measurement noise
A = 1.0
H = 1.0



friction_estimate = np.full(NUM_WHEELS, 0.8)
P = np.ones(NUM_WHEELS)
# Calculate slip ratio
def calculate_slip_ratio(vehicle_speed, wheel_speed):
    if vehicle_speed == 0:
        return 0
    return (vehicle_speed - wheel_speed) / vehicle_speed








In [ ]:

# Defining the model parameters
net = cv2.dnn.readNet('/home/vivekp22/Desktop/Pothole_github/pothole-detection-main/project_files/yolov4_tiny.weights', '/home/vivekp22/Desktop/Pothole_github/pothole-detection-main/project_files/yolov4_tiny.cfg')
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)
pothole_model = cv2.dnn_DetectionModel(net)
pothole_model.setInputParams(size=(480, 480), scale=1/255, swapRB=True)
input_video_path = '/home/vivekp22/Downloads/AdobeStock_Dry_Asphalt.mov' # Replace with the path to your input video
output_video_path = '/home/vivekp22/Desktop/PT_Final/pt3/27jan_1_Asph2.mp4' # Replace with the path to your output video
dashboard_image_path = '/home/vivekp22/Downloads/Dashboard_images_Final.png'  # Replace with the path to your dashboard image

# Load the dashboard image
dashboard = cv2.imread(dashboard_image_path, cv2.IMREAD_UNCHANGED)

# Check if the image has 4 channels (RGBA)
if dashboard.shape[-1] == 4:
    # Convert the image from RGBA to RGB
    dashboard = cv2.cvtColor(dashboard, cv2.COLOR_BGRA2BGR)

# Resize the dashboard to fit the video frame (adjust dimensions as needed)
dashboard_height, dashboard_width = 180, 420  # Adjust the size if needed
dashboard = cv2.resize(dashboard, (dashboard_width, dashboard_height))

cap = cv2.VideoCapture(input_video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
# fps=2
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))



out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Font settings for displaying text
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.5
font_color = (255, 255, 255)  # White text
thickness = 1

vehicle_speed = 20.0  # m/s
wheel_speeds = [19.0, 18.5, 19.2, 18.8]


In [ ]:

import cv2
import time
import numpy as np


class PotholeBuffer:
    def __init__(self, high_hold_time=2, medium_hold_time=2, low_hold_time=1):
        self.high_hold_time = high_hold_time
        self.medium_hold_time = medium_hold_time
        self.low_hold_time = low_hold_time
        self.buffer = []
        self.max_pothole_type = None
        self.friction_potholes = None

    def update_buffer(self, max_pothole_type, friction_potholes):
        self.buffer.append((max_pothole_type, friction_potholes))
        if max_pothole_type == "High":
            time.sleep(self.high_hold_time)
        elif max_pothole_type == "Medium":
            time.sleep(self.medium_hold_time)
        else:
            time.sleep(self.low_hold_time)
        self.buffer = self.buffer[-60:] # Assuming 60 frames per second
        max_count = 0
        max_pothole_type_detected = None
        for pothole_type, _ in self.buffer:
            if pothole_type in ["High", "Medium", "Low"]:
                count = self.buffer.count((pothole_type, _))
                if count > max_count:
                    max_count = count
                    max_pothole_type_detected = pothole_type
        self.max_pothole_type = max_pothole_type_detected
        if max_pothole_type_detected == "High":
            self.friction_potholes = 0.2
        elif max_pothole_type_detected == "Medium":
            self.friction_potholes = 0.3
        elif max_pothole_type_detected == "Low":
            self.friction_potholes = 0.4

    def get_max_pothole_type(self):
        return self.max_pothole_type

    def get_friction_potholes(self):
        return self.friction_potholes

def detect_pothole(frame):
    # This function is a placeholder to detect pothole using pothole_model.detect
    return "High" #fake detected pothole type

# Create an instance of the PotholeBuffer class.
pothole_buffer = PotholeBuffer()

# Analyze the frame with detection model
frame_pothole_dict = {"Low": 0, "Medium": 0, "High": 0}
total_pothole_areas = {"Low": 0, "Medium": 0, "High": 0}

cap = cv2.VideoCapture('/home/vivekp22/Downloads/AdobeStock_Dry_Asphalt.mov')

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame[0:dashboard_height, 0:dashboard_width] = dashboard

    # Analysis the stream with detection model
    classes, scores, boxes = pothole_model.detect(frame, 0.5, 0.4)

    for (classid, score, box) in zip(classes, scores, boxes):
        label = "pothole"
        x, y, w, h = box
        recarea = w*h
        area = frame_width*frame_height

        if(len(scores)!=0 and scores[0]>=0.5):
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0,0,255), 1)

            if recarea <= (area / 100) * 0.1:
                type = "Low"
            elif (area / 100) * 0.3 < recarea <= (area / 100) * 0.7:
                type = "Medium"
            else:
                type = "High"

            # Update pothole dictionary
            if type in frame_pothole_dict:
                frame_pothole_dict[type] += 1
                total_pothole_areas[type] += recarea

    if sum(frame_pothole_dict.values()) == 0:
        # No Potholes
        resized_frame1 = cv2.resize(frame, (224, 224))
        resized_frame1 = cv2.cvtColor(resized_frame1, cv2.COLOR_BGR2RGB)
        normalized = resized_frame1 / 255.0
        frame_array = np.array([normalized])
        predictions = model.predict(frame_array)
        class_id = np.argmax(predictions)
        predicted_class = class_names[class_id]

        predicted_class_indices = np.argsort(predictions[0])[::-1][:3]
        predicted_class_names = [class_names[index] for index in predicted_class_indices]
        confidences = [predictions[0, index] for index in predicted_class_indices]

        slip_ratios = [calculate_slip_ratio(vehicle_speed, tire_angular_velocity)
                        for _ in range(len(predicted_class_names))]

        mu_values = []
        for class_name in predicted_class_names:
            mu_range = friction_coefficients[class_name].split('-')
            mu = (float(mu_range[0]) + float(mu_range[1])) / 2 if len(mu_range) == 2 else float(mu_range[0])
            mu_values.append(mu)

        sum_confidences = sum(confidences)
        friction = sum((conf / sum_confidences) * mu for conf, mu in zip(confidences, mu_values))
        surface_friction = f'{friction:.2f}'


        cv2.putText(
          frame,
          f"Special Feature: None",
          (115, 45),
          font, 0.3, (0, 255, 0), thickness, cv2.LINE_AA
        )

        cv2.putText(
            frame,
            f"WFL: {predicted_class}",
            (10, 20),
            font, font_scale, font_color, thickness, cv2.LINE_AA
        )
        cv2.putText(
            frame,
            f"Mu: {surface_friction}",
            (33, 23 + int(font_scale * 25)),
            font, font_scale, font_color, thickness, cv2.LINE_AA
        )

        cv2.putText(
            frame,
            f"WFR: {predicted_class}",
            (dashboard_width - 150, 20),
            font, font_scale, font_color, thickness, cv2.LINE_AA
        )
        cv2.putText(
            frame,
            f"Mu: {surface_friction}",
            (dashboard_width - 120, 23 + int(font_scale * 25)),
            font, font_scale, font_color, thickness, cv2.LINE_AA
        )

        cv2.putText(
            frame,
            f"WRL: {predicted_class}",
            (12, dashboard_height - 20),
            font, font_scale, font_color, thickness, cv2.LINE_AA
        )
        cv2.putText(
            frame,
            f"Mu: {surface_friction}",
            (35, dashboard_height - 17 + int(font_scale * 25)),
            font, font_scale, font_color, thickness, cv2.LINE_AA
        )

        cv2.putText(
            frame,
            f"WRR: {predicted_class}",
            (dashboard_width - 150, dashboard_height - 20),
            font, font_scale, font_color, thickness, cv2.LINE_AA
        )
        cv2.putText(
            frame,
            f"Mu: {surface_friction}",
            (dashboard_width - 120, dashboard_height - 17 + int(font_scale * 25)),
            font, font_scale, font_color, thickness, cv2.LINE_AA
        )
    else:

        max_pothole_type_in_frame = max(total_pothole_areas, key=total_pothole_areas.get)

        pothole_buffer.update_buffer(max_pothole_type_in_frame, max_pothole_type_in_frame)

        max_pothole_detected = pothole_buffer.get_max_pothole_type()
        friction_potholes = pothole_buffer.get_friction_potholes()

        cv2.putText(
          frame,
          f"Special Feature: Pothole Detected: {max_pothole_detected}",
          (80, 45),
          font, 0.3, (0, 255, 0), thickness, cv2.LINE_AA
        )

        cv2.putText(
            frame,
            f"WFL: {predicted_class}",
            (10, 20),
            font, font_scale, font_color, thickness, cv2.LINE_AA
        )
        cv2.putText(
            frame,
            f'Mu: {friction_potholes:.2f}',
            (33, 23 + int(font_scale * 25)),
            font, font_scale, font_color, thickness, cv2.LINE_AA
        )

        cv2.putText(
            frame,
            f"WFR: {predicted_class}",
            (dashboard_width - 150, 20),
            font, font_scale, font_color, thickness, cv2.LINE_AA
        )
        cv2.putText(
            frame,
            f'Mu: {friction_potholes:.2f}',
            (dashboard_width - 120, 23 + int(font_scale * 25)),
            font, font_scale, font_color, thickness, cv2.LINE_AA
        )

        cv2.putText(
            frame,
            f"WRL: {predicted_class}",
            (12, dashboard_height - 20),
            font, font_scale, font_color, thickness, cv2.LINE_AA
        )
        cv2.putText(
            frame,
            f'Mu: {friction_potholes:.2f}',
            (35, dashboard_height - 17 + int(font_scale * 25)),
            font, font_scale, font_color, thickness, cv2.LINE_AA
        )

        cv2.putText(
            frame,
            f"WRR: {predicted_class}",
            (dashboard_width - 150, dashboard_height - 20),
            font, font_scale, font_color, thickness, cv2.LINE_AA
        )
        cv2.putText(
            frame,
            f'Mu: {friction_potholes:.2f}',
            (dashboard_width - 120, dashboard_height - 17 + int(font_scale * 25)),
            font, font_scale, font_color, thickness, cv2.LINE_AA
        )

    out.write(frame)

cap.release()
out.release()

print(f'Output video saved to: {output_video_path}')